In [1]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import pandas_datareader as pdr
import pandas_datareader.wb as wb
from jupyter_dash import JupyterDash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
from datetime import datetime
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
from datetime import datetime as dt
import datetime
from datetime import date

import warnings
warnings.filterwarnings("ignore") # Hiding unecessary warnings to make the notebook more readable

/var/folders/kr/tz8q1r495dz1w5_78501tbw80000gn/T/ipykernel_7852/2673247886.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/kr/tz8q1r495dz1w5_78501tbw80000gn/T/ipykernel_7852/2673247886.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/kr/tz8q1r495dz1w5_78501tbw80000gn/T/ipykernel_7852/2673247886.py:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [2]:
# Loading the data and filtering for the columns we will use

df_raw = pd.read_csv('EXR.csv', sep=';')

cols = ['BASE_CUR', 'Base Currency', 'Unit Multiplier', 'TIME_PERIOD', 'OBS_VALUE']

df = df_raw[cols]

### Feature 1: Currency table

In [3]:
# Filtering data for 16th and 19th of May, the last 2 days of observations
df_last2= df[df['TIME_PERIOD'].isin(['2023-05-16', '2023-05-19'])]

# Pivot data to get each date as a column
df_pivot = df_last2.pivot(index=['BASE_CUR', 'Base Currency'], columns='TIME_PERIOD', values='OBS_VALUE')

# Resetting index to make 'BASE_CUR' and 'Base Currency' columns again
df_pivot.reset_index(inplace=True)

# Renaming columns
df_pivot.columns = ['Code', 'Currency', '16 May', '19 May']

# Reordering columns to mirror Norges Bank
df_pivot = df_pivot[['Currency', 'Code', '16 May', '19 May']]

In [4]:
# Creating our currency table as a card

table_card = dbc.Card(
    
    children=[
        
        html.H4('Most Recent Exchange Rates:'),
        html.Br(),
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df_pivot.columns],
            data=df_pivot.to_dict('records'),
            
            style_table={'overflowX': 'auto', 'maxWidth': '80%'},
            style_data={
                'width': '7px',
                'maxWidth': '7px',
                'minWidth': '7px',
                'overflow': 'hidden',
                'textOverflow': 'ellipsis',
                'font-family': 'Montserrat',
                'color': 'rgb(34, 72, 112)'
            },
            # Aligning our text columns to the left
            style_cell_conditional=[
                {
                    'if': {'column_id': ['Currency', 'Code']},
                    'textAlign': 'left'
                },
            ],
            
            style_data_conditional=[
                { # Adding shade to every other row for readibility
                    'if': {'row_index': 'even'},
                    'backgroundColor': 'rgb(220, 220, 220)'
                },
                # Adding indentations and column width
                {
                    'if': {'column_id': 'Currency'},
                    'width': '20px',
                    'maxWidth': '20px',
                    'minWidth': '20px',
                    'padding-left': '8px',
                },
                {
                    'if': {'column_id': 'Code'},
                    'width': '10px',
                    'maxWidth': '10px',
                    'minWidth': '10px',
                    'padding-left': '8px',
                },
                {
                    'if': {'column_id': '16 May'},
                    'width': '10px',
                    'maxWidth': '10px',
                    'minWidth': '10px',
                    'padding-right': '8px', 
                },
                {
                    'if': {'column_id': '19 May'},
                    'width': '10px',
                    'maxWidth': '10px',
                    'minWidth': '10px',
                    'padding-right': '8px', 
                },

            ],
            # Styling our header row
            style_header={
                'color': 'black',
                'fontWeight': '900',  
                'font-family': 'Montserrat',
                'fontSize': '18px',
                'margin': '30px 0',
                'padding-left': '8px',  
                'padding-right': '8px',  
            }
        ),
    ],
    body=True,
)


### Feature 2: Currency converter

In [5]:
# New column adding currency code in parenthesis
df['Currency'] = df['Base Currency'] + ' (' + df['BASE_CUR'] + ')'

# Dropdown for choosing which currency to convert
base_currency_dropdown = dcc.Dropdown(
    id='base-currency-dropdown',
    options=[{'label': i, 'value': i} for i in df['Currency'].unique()],
    value=df['Currency'].iloc[0]
)

# The amount to convert
amount_input = dcc.Input(
    id='amount-input',
    type='number',
    value=1
)

# Container for the conversion result
conversion_result = html.Div(id='conversion-result')

# Defining feature layout
currency_converter = dbc.Card(
    children=[
        html.H4('Currency Converter'),
        dbc.Row([
            dbc.Col([
                html.P('Enter amount to convert:'),
                amount_input
            ], width=3),
            dbc.Col([
                html.P('Select a base currency:'),
                base_currency_dropdown
            ], width=3),
            dbc.Col([
                html.P('Conversion result:'),
                conversion_result
            ], width=4),
        ])
    ],
    body=True,
)

### Feature 3: Plot

In [6]:
# Converting date column to pd.datetime format
df['TIME_PERIOD'] = pd.to_datetime(df['TIME_PERIOD'])

# Converting currency exhange to numeric
df['OBS_VALUE'] = pd.to_numeric(df['OBS_VALUE'], errors='coerce')
df['Month'] = df['TIME_PERIOD'].dt.to_period('M')
df['Year'] = df['TIME_PERIOD'].dt.to_period('Y')

# Create a dataframe with average monthly exchange rate for each currency
monthly_df = df.groupby(['Currency', 'Month'])['OBS_VALUE'].mean().reset_index()

# Create a dataframe with average annual exchange rate for each currency
annual_df = df.groupby(['Currency', 'Year'])['OBS_VALUE'].mean().reset_index()


# Create a date picker for each tab, to avoid error

# Daily time picker, default range to be month to date
daily_time_period_picker = dcc.DatePickerRange(
    id='daily-time-period-picker',
    min_date_allowed=df['TIME_PERIOD'].min(),
    max_date_allowed=df['TIME_PERIOD'].max(),
    initial_visible_month=df['TIME_PERIOD'].max(),
    start_date=date(2023, 5, 1),
    end_date=df['TIME_PERIOD'].max()
)

# Monthly time picker, default range to be the last 12 months
monthly_time_period_picker = dcc.DatePickerRange(
    id='monthly-time-period-picker',
    min_date_allowed=df['TIME_PERIOD'].min(),
    max_date_allowed=df['TIME_PERIOD'].max(),
    initial_visible_month=df['TIME_PERIOD'].max(),
    start_date=df['TIME_PERIOD'].max() - pd.DateOffset(months=12),
    end_date=df['TIME_PERIOD'].max()
)

# Annual time picker, default range to be the last 10 years
annual_time_period_picker = dcc.DatePickerRange(
    id='annual-time-period-picker',
    min_date_allowed=df['TIME_PERIOD'].min(),
    max_date_allowed=df['TIME_PERIOD'].max(),
    initial_visible_month=df['TIME_PERIOD'].max(),
    start_date=df['TIME_PERIOD'].max() - pd.DateOffset(years=10),
    end_date=df['TIME_PERIOD'].max()
)



# Daily graph
daily_graph = dcc.Graph(id='daily-exchange-rate-graph')

# Daily plot
daily_plot = dbc.Card(
    children=[
        
        html.P('Select a time period:'),
        daily_time_period_picker,
        daily_graph
    ],
    body=True,
)

# Monthly graph
monthly_graph = dcc.Graph(id='monthly-exchange-rate-graph')

# Monthly Plot
monthly_plot = dbc.Card(
    children=[
        
        html.P('Select a time period:'),
        monthly_time_period_picker,
        monthly_graph
    ],
    body=True,
)


# Yearly graph
yearly_graph = dcc.Graph(id='annual-exchange-rate-graph')

# Yearly Plot
yearly_plot = dbc.Card(
    children=[
        
        html.P('Select a time period:'),
        annual_time_period_picker,
        yearly_graph
    ],
    body=True,
)

In [7]:
# Function for generating plots

def generate_plot(dataframe, x_col, y_col, title):
    fig = go.Figure(data=go.Scatter(x=dataframe[x_col], 
                                    y=dataframe[y_col], 
                                    mode='lines+markers', 
                                    marker=dict(color='rgb(17, 82, 151)'), 
                                    line=dict(width=2),  
                                    hovertemplate=  
                                        '<b>Date</b>: %{x}' +
                                        '<br><b>Exchange Rate</b>: %{y}'))

    fig.update_layout(
        title=title,
        title_x=0.5,  
        title_font=dict(size=24),  
        xaxis_title='Date',
        yaxis_title='Exchange Rate',
        plot_bgcolor='rgb(243, 243, 243)', 
        xaxis=dict(
            title_font=dict(size=18),  
        ),
        yaxis=dict(
            title_font=dict(size=18),  
        ),
        hovermode="x unified",  
    )

    return fig

### App layout:

In [8]:
# Loading montserrat css file, as it is a very crisp font
montserrat = 'https://fonts.googleapis.com/css2?family=Montserrat&display=swap'

# Will be using montserrat fonts in addition to the PULSE theme
app = JupyterDash(external_stylesheets = [dbc.themes.PULSE, montserrat])

app.layout = html.Div(style={'font-family': 'Montserrat', 'padding': '0 10%'}, children=[
    
    dbc.Row([
        dbc.Col(html.Img(src='/assets/nhh.png', style={'height':'50%', 'width':'70%'}), width=2),
        dbc.Col(html.H1('Currency dashboard', style={'font-family': 'Montserrat'}), width=8),
        dbc.Col(html.H3('BAN438 | Candidate: 33', style={'textAlign': 'right', 'font-family': 'Montserrat'}), width=2),
    ], style={'margin': '0px'}),
    
    html.Br(),

    dbc.Tabs(style={'font-family': 'Montserrat'}, children=[
        dbc.Tab(table_card, label="All Exchange rates"),
        dbc.Tab([
            currency_converter,
            html.Br(),
            dbc.Tabs(id='timescale-tabs', children=[
                dbc.Tab(daily_plot, label="Daily"),
                dbc.Tab(monthly_plot, label="Monthly"),
                dbc.Tab(yearly_plot, label="Annual")
            ]),
        ], label="Choose Currency"),
    ]),
])

# Define the callback to update the amount input
@app.callback(
    Output('amount-input', 'value'),
    Input('base-currency-dropdown', 'value')
)
def update_amount_input(selected_currency):
    unit_multiplier = df[df['Currency'] == selected_currency]['Unit Multiplier'].iloc[0]
    if unit_multiplier == 'Hundreds':
        return 100
    else:
        return 1

# Callback to perform the conversion
@app.callback(
    Output('conversion-result', 'children'),
    [Input('base-currency-dropdown', 'value'),
    Input('amount-input', 'value')]
)
def perform_conversion(selected_currency, amount):
    # Get the most recent exchange rate for the selected currency
    exchange_rate = df[df['Currency'] == selected_currency]['OBS_VALUE'].iloc[-1]
    # Perform the conversion
    converted_amount = amount * exchange_rate
    # Return the result with custom inline styling
    return html.P([
        f'{amount} {selected_currency} = ',
        html.Span(f'{converted_amount:.2f} ', style={'font-weight': 'bold', 'text-decoration': 'underline'}),
        html.Span('NOK', style={'font-weight': 'bold', 'text-decoration': 'underline'})
    ])

# Callback for daily plot
@app.callback(
    Output('daily-exchange-rate-graph', 'figure'),
    [Input('base-currency-dropdown', 'value'),
     Input('daily-time-period-picker', 'start_date'),
     Input('daily-time-period-picker', 'end_date')]
)

def update_daily_plot(selected_currency, start_date, end_date):
    # Filter the dataframe based on the selected currency and time period
    filtered_df = df[(df['Currency'] == selected_currency) & 
                     (df['TIME_PERIOD'] >= start_date) & 
                     (df['TIME_PERIOD'] <= end_date)]

    # Generate and return the plot
    return generate_plot(filtered_df, 'TIME_PERIOD', 'OBS_VALUE', 'Daily Exchange Rate Over Time')

@app.callback(
    Output('monthly-exchange-rate-graph', 'figure'),
    [Input('base-currency-dropdown', 'value'),
     Input('monthly-time-period-picker', 'start_date'),
     Input('monthly-time-period-picker', 'end_date')]
)
def update_monthly_plot(selected_currency, start_date, end_date):
    # Convert string dates to datetime
    start_date = dt.strptime(start_date[:10], '%Y-%m-%d')
    end_date = dt.strptime(end_date[:10], '%Y-%m-%d')
    # Convert pandas Period to datetime
    monthly_df['Month_start'] = monthly_df['Month'].apply(lambda m: m.start_time)
    monthly_filtered = monthly_df[(monthly_df['Currency'] == selected_currency) & 
                                  (monthly_df['Month_start'] >= start_date) & 
                                  (monthly_df['Month_start'] <= end_date)]

    return generate_plot(monthly_filtered, 'Month_start', 'OBS_VALUE', 'Monthly Exchange Rate Over Time')

# Callback for annual plot
@app.callback(
    Output('annual-exchange-rate-graph', 'figure'),
    [Input('base-currency-dropdown', 'value'),
     Input('annual-time-period-picker', 'start_date'),
     Input('annual-time-period-picker', 'end_date')])
def update_annual_plot(selected_currency, start_date, end_date):
    start_date = dt.strptime(start_date[:10], '%Y-%m-%d')
    end_date = dt.strptime(end_date[:10], '%Y-%m-%d')
    
    annual_df['Year_start'] = annual_df['Year'].apply(lambda y: y.start_time)
    annual_filtered = annual_df[(annual_df['Currency'] == selected_currency) & 
                                (annual_df['Year_start'] >= start_date) & 
                                (annual_df['Year_start'] <= end_date)]

    return generate_plot(annual_filtered, 'Year_start', 'OBS_VALUE', 'Yearly Exchange Rate Over Time')


app.run_server(port=5891)


Dash app running on http://127.0.0.1:5891/
